In [1]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so
import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|___________________________|



In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
# train
# test
# sample_submission

In [4]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [5]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
df

,facts
0,phil amant caidate for public office made tele...
1,ramon nelson was ridin his bike when he suffer...
2,an alabama ate court convicted billy joe mawoo...
3,victor linkletter was convicted in ate court o...
4,in selma alabama an intruder broke into apartm...
...,...
2473,conress ameed clean air act rouh enery policy ...
2474,alliance bo fu inc an invement fu purchased ap...
2475,in dirict court sentenced manuel peuero to mon...
2476,enrico cyr lawful permanent resident pled uilt...


In [6]:
train_cleansed = so.law_train_clean(train, df)
train_cleansed

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,phil amant caidate for public office made tele...,1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,ramon nelson was ridin his bike when he suffer...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",an alabama ate court convicted billy joe mawoo...,1
3,TRAIN_0003,Linkletter,Walker,victor linkletter was convicted in ate court o...,0
4,TRAIN_0004,William Earl Fikes,Alabama,in selma alabama an intruder broke into apartm...,1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",conress ameed clean air act rouh enery policy ...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",alliance bo fu inc an invement fu purchased ap...,1
2475,TRAIN_2475,Peguero,United States,in dirict court sentenced manuel peuero to mon...,0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,enrico cyr lawful permanent resident pled uilt...,0


## BERT


In [7]:
'''
@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}
'''

'\n@article{turc2019,\n  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},\n  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},\n  journal={arXiv preprint arXiv:1908.08962v2 },\n  year={2019}\n}\n'

In [8]:
train_facts = pd.DataFrame(train_cleansed['facts'])

In [12]:
# combined_se_df = pd.DataFrame()
# for fact in tqdm(train_facts['facts']):
#     encoded_input = tokenizer(fact, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
#     df = pd.DataFrame(encoded_input.items())
#     combined_se_df = pd.concat([combined_se_df, df])
    

# combined_se_df = combined_se_df.rename(columns={0:'tbd', 1:'hmm'})
# combined_se_df
model_input_df = dlc.bert_tokenizer(train_facts, 'facts')
# train_bert_tokenized.to_csv("./train_bert_tokenized.csv", index=False)

100%|█████████████████████████████████████| 2478/2478 [00:01<00:00, 2115.62it/s]


In [13]:
model_input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2478 non-null   object
dtypes: object(1)
memory usage: 19.5+ KB


In [15]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased", add_pooling_layer=False, output_hidden_states=True, output_attentions=True)
model.num_parameters()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


108891648

In [ ]:

output_total_list = []
for index in tqdm(range(len(model_input_df))):
    output = model(**model_input_df.iloc[index, 0])
#     words_embedding = model.embeddings.word_embeddings
#     position_embedding = model.embeddings.position_embeddings
#     token_type_embedding = model.embeddings.token_type_embeddings
    output_total_list.append(output)

output_df = pd.DataFrame(output_total_list)


  5%|█▊                                      | 114/2478 [01:13<40:30,  1.03s/it]

In [ ]:
# 3D to 2D

attention_mask_df = dlc.tensor_2_2d(train_bert_tokenized, 0)
input_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 1)
token_type_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 2)

attention_mask_df.info()
print('\n _______________________________ \n')
input_ids_df.info()
print('\n _______________________________ \n')
token_type_ids_df.info()


In [ ]:
# attention_mask_df.info()
attention_mask_df
# input_ids_df.info()
# input_ids_df
# token_type_ids_df.info()
# token_type_ids_df


In [ ]:
temp = pd.DataFrame()
temp = pd.concat([train_cleansed['ID'], attention_mask_df], axis=1)
temp = pd.concat([temp, input_ids_df], axis=1)
train_BertToken_df = pd.concat([temp, token_type_ids_df], axis=1)
train_BertToken_df


In [ ]:
tBTdf = so.right_merger(train_cleansed, train_BertToken_df, 0)
tBTdf